In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/estimation-of-obesity-levels-uci-dataset/ObesityDataSet_raw_and_data_sinthetic.arff
/kaggle/input/estimation-of-obesity-levels-uci-dataset/ObesityDataSet_raw_and_data_sinthetic.csv


In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/estimation-of-obesity-levels-uci-dataset/ObesityDataSet_raw_and_data_sinthetic.csv')
df.head()


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:

print(df.isnull().sum())

df.info()


Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height             

In [4]:
print(df['CALC'].unique())



['no' 'Sometimes' 'Frequently' 'Always']


In [5]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(categories=[['no', 'Sometimes', 'Frequently', 'Always']])
df['CALC'] = ordinal_encoder.fit_transform(df[['CALC']])



In [6]:
print(df['CALC'].unique())


[0. 1. 2. 3.]


In [7]:
nominal_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']
df = pd.get_dummies(df, columns=nominal_cols)


In [8]:
print(df.columns.tolist())


['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CAEC', 'CH2O', 'FAF', 'TUE', 'CALC', 'NObeyesdad', 'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no', 'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes', 'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']


In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['NObeyesdad'] = label_encoder.fit_transform(df['NObeyesdad'])


In [10]:
print(df.select_dtypes(include='object').columns)


Index(['CAEC'], dtype='object')


In [11]:
print(df['CAEC'].unique())


['Sometimes' 'Frequently' 'Always' 'no']


In [12]:
df['CAEC'] = df['CAEC'].str.strip().str.lower().str.capitalize()

from sklearn.preprocessing import OrdinalEncoder

caec_encoder = OrdinalEncoder(categories=[['No', 'Sometimes', 'Frequently', 'Always']])
df['CAEC'] = caec_encoder.fit_transform(df[['CAEC']])


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9479905437352246
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        56
           1       0.86      0.90      0.88        62
           2       0.99      0.95      0.97        78
           3       0.97      0.98      0.97        58
           4       1.00      1.00      1.00        63
           5       0.89      0.88      0.88        56
           6       0.96      0.96      0.96        50

    accuracy                           0.95       423
   macro avg       0.95      0.95      0.95       423
weighted avg       0.95      0.95      0.95       423



In [14]:
def predict_obesity_and_risk(model, user_input):
    from numpy import array
    
    probas = model.predict_proba([array(user_input)])[0]
    pred_class = probas.argmax()

    class_labels = [
        'Insufficient_Weight',
        'Normal_Weight',
        'Overweight_Level_I',
        'Overweight_Level_II',
        'Obesity_Type_I',
        'Obesity_Type_II',
        'Obesity_Type_III'
    ]

    risk_map = {
        0: 15,
        1: 25,
        2: 50,
        3: 65,
        4: 80,
        5: 90,
        6: 97
    }

    risk_percent = sum(p * risk_map[i] for i, p in enumerate(probas))
    label = class_labels[pred_class]

    return label, round(risk_percent, 2)


In [15]:
sample_input = X_test.iloc[0].tolist()

label, risk = predict_obesity_and_risk(model, sample_input)

print("Predicted Category:", label)
print("Obesity Risk (%):", risk)


Predicted Category: Insufficient_Weight
Obesity Risk (%): 23.76


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [16]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [17]:
import joblib

joblib.dump(model, 'obesity_model.pkl')


['obesity_model.pkl']

In [18]:
import zipfile

with zipfile.ZipFile('obesity_model.zip', 'w') as zipf:
    zipf.write('obesity_model.pkl')


In [19]:
from IPython.display import FileLink
FileLink('obesity_model.zip')


/kaggle/working/obesity_model.zip